<a href="https://colab.research.google.com/github/GabrielAzpitarte/Machine-learning-projects/blob/main/LSTM_name_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notre premier LSTM
On va utiliser nos propres données de prénoms

In [ ]:
liste_prenoms = ["Benjamin", "Danylo", "Paul", "Ethan", "Hazem", "Noé", "Raphaël", "Karliss", "Gabriel",
                 "Timothée","Soizic","Adrien","Théo","Maeva","Samy","Camille","Romain","Alexandre","Arthur"]

##Mise en forme des données
On va faire une chaîne de caractère qui joint tous les éléments de la liste.

Pour ce faire, on va utiliser la méthode str.join(list)

In [ ]:
#On commence par chercher la longueur des séquences
longueur_sequence = 0

#Il va falloir mettre cette longueur par rapport au plus long nom
for prenom in liste_prenoms :
  longueur = len(prenom)

  #si la longueur du prénom est plus grande que celle de la séquence, on update cette dernière
  if longueur > longueur_sequence :
    longueur_sequence = longueur

#On ajoute 1 pour prendre en compte le séparateur ' '
longueur_sequence += 1

print("Longueur des séquences :", longueur_sequence)

Longueur des séquences : 10


In [ ]:
#On peut maintenant faire notre chaîne de prénoms :
chaine_prenoms = ' '.join(liste_prenoms).lower()

#On ajoute un espace à la fin
chaine_prenoms += ' '

print("Chaîne des prénoms :", chaine_prenoms)

Chaîne des prénoms : benjamin danylo paul ethan hazem noé raphaël karliss gabriel timothée soizic adrien théo maeva samy camille romain alexandre arthur 


In [ ]:
print(chaine_prenoms.capitalize())

Benjamin danylo paul ethan hazem noé raphaël karliss gabriel timothée soizic adrien théo maeva samy camille romain alexandre arthur 


On va faire une liste des différentes lettres et on pourra ainsi remplacer le caractère dans chaine_prenom par son indice.

In [ ]:
liste_lettres = []

#Pour chaque lettre de la chaine, on ajoute la lettre à liste_lettres si elle n'est pas déjà dedans
for lettre in chaine_prenoms :
  if lettre not in liste_lettres :
    liste_lettres.append(lettre)

print("Nombre de caractères différents :",len(liste_lettres))
print(liste_lettres)
print('indice de la lettre h :', liste_lettres.index('h'))

Nombre de caractères différents : 26
['b', 'e', 'n', 'j', 'a', 'm', 'i', ' ', 'd', 'y', 'l', 'o', 'p', 'u', 't', 'h', 'z', 'é', 'r', 'ë', 'k', 's', 'g', 'c', 'v', 'x']
indice de la lettre h : 15


On va pouvoir créer les séquences glissées sur la chaîne de prénoms

In [ ]:
#On va créer les séquences et la lettre suivante en même temps
sequences = []
lettres_suivantes = []

#On récupère la longueur de la chaîne de prénoms :
longueur_chaine = len(chaine_prenoms)

#on va faire une boucle sur des indices entre 0 et longueur_chaine - longueur_sequence :
for indice in range(0, longueur_chaine - longueur_sequence) :
  #on récupère une séquence dans chaîne nombre située entre indice et indice + longueur_sequence
  sequence = chaine_prenoms[indice : indice + longueur_sequence]
  #on récupère la lettre suivante :
  lettre_suivante = chaine_prenoms[indice + longueur_sequence]

  #On ajoute ces derniers éléments à nos 2 listes
  sequences.append(sequence)
  lettres_suivantes.append(lettre_suivante)

print("Nombre de séquences :", len(sequences))
#Vérification sur les 10 premiers éléments et le dernier
#On utilise enumerate() pour récupérer l'indice et l'élément de la liste avec for
for indice,sequence in enumerate(sequences[:10]):
  print("Séquence :", sequence, "    Lettre suivante :",lettres_suivantes[indice])

print()
print("Dernières occurences :")
print("Séquence :", sequences[-1], "    Lettre suivante :",lettres_suivantes[-1])

Nombre de séquences : 122
Séquence : benjamin d     Lettre suivante : a
Séquence : enjamin da     Lettre suivante : n
Séquence : njamin dan     Lettre suivante : y
Séquence : jamin dany     Lettre suivante : l
Séquence : amin danyl     Lettre suivante : o
Séquence : min danylo     Lettre suivante :  
Séquence : in danylo      Lettre suivante : p
Séquence : n danylo p     Lettre suivante : a
Séquence :  danylo pa     Lettre suivante : u
Séquence : danylo pau     Lettre suivante : l

Dernières occurences :
Séquence : dre arthur     Lettre suivante :  


## Création des X, Y

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

#On stocke la longueur de liste_lettres
categories = len(liste_lettres)

#on va commencer par créer les X
X=[]

#on crée une boucle sur les séquences
for sequence in sequences :
  #on va créer une liste vide pour récolter les indices des caractères
  liste = []
  for caractere in sequence :
    #on récupère l'indice correspondant au caractère
    indice = liste_lettres.index(caractere)
    #on ajoute l'indice à la liste
    liste.append(indice)

  #on convertit tout de suite avec to_categorical pour avoir un vecteur de proba de 26 éléments
  liste = to_categorical(liste,categories)
  #On ajoute la liste dans X
  X.append(liste)

X= np.asarray(X)
print(X.shape)

#pour les Y, on ne change pas de méthode
#il faut quand même convertir les caractères en indice
Y=[]
for caractere in lettres_suivantes :
  indice = liste_lettres.index(caractere)
  Y.append(indice)

#to_categorical
Y = to_categorical(Y,categories)

print(Y.shape)

(122, 10, 26)
(122, 26)


##Réseau de neurones avec LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

modele = Sequential()

#On ajoute en entrée une couche de neurones LSTM
modele.add(LSTM(64, input_shape = (longueur_sequence,categories), recurrent_dropout = 0.1))

#On ajoute la sortie
modele.add(Dense(categories, activation='softmax'))

modele.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                23296     
                                                                 
 dense (Dense)               (None, 26)                1690      
                                                                 
Total params: 24,986
Trainable params: 24,986
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import RMSprop

optimizer = RMSprop(learning_rate = 0.01)

modele.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',])

modele.fit(X, Y, batch_size = 8, epochs = 30)

modele.save('name_generator.h5')

Epoch 1/30
16/16 [==============================] - 7s 27ms/step - loss: 3.1303 - accuracy: 0.0902
Epoch 2/30
16/16 [==============================] - 0s 27ms/step - loss: 2.9120 - accuracy: 0.1803
Epoch 3/30
16/16 [==============================] - 0s 24ms/step - loss: 2.8332 - accuracy: 0.1967
Epoch 4/30
16/16 [==============================] - 0s 28ms/step - loss: 2.6608 - accuracy: 0.2623
Epoch 5/30
16/16 [==============================] - 0s 26ms/step - loss: 2.4674 - accuracy: 0.2787
Epoch 6/30
16/16 [==============================] - 0s 23ms/step - loss: 2.2861 - accuracy: 0.3279
Epoch 7/30
16/16 [==============================] - 0s 20ms/step - loss: 2.0249 - accuracy: 0.3607
Epoch 8/30
16/16 [==============================] - 0s 20ms/step - loss: 1.8747 - accuracy: 0.3934
Epoch 9/30
16/16 [==============================] - 0s 20ms/step - loss: 1.6677 - accuracy: 0.4836
Epoch 10/30
16/16 [==============================] - 0s 20ms/step - loss: 1.4754 - accuracy: 0.5000
Epoch 11/

##Création du prédicteur

In [ ]:
from random import choice

#On va décider de la première lettre :
#lettre = 'é'
lettre = choice(liste_lettres)

#On récupère la dernière séquence
sequence="dre arthur"

#On ajoute à la fin un caractère espace et la lettre choisie et on supprime les 2 premiers caractères
sequence += ' ' + lettre
sequence = sequence[2:]

#print(len(sequence),sequence)

#On crée une nouvelle chaine pour stocker les prénoms générés en commençant par la lettre choisie :
prenoms_generes = lettre

#On donne combien de noms vont être générés :
nombre_noms = 10

while nombre_noms > 0 :
  X = []

  #on recrée X
  for caractere in sequence :
    indice = liste_lettres.index(caractere)
    X.append(indice)

  #On to_categorical X
  X = to_categorical(X, categories)

  #il faut reshape X pour avoir un tenseur en 3 dimensions
  X = np.reshape(X, (1,X.shape[0],X.shape[1]))
  #print(X.shape)

  #On prédit le vecteur de probabilités
  probas = modele.predict(X, verbose = 0)
  probas = probas[0]

  #On choisit aléatoirement l'indice parmi les catégories en utilisant la distribution calculée
  indice = np.random.choice(categories, p=probas)

  #On récupère la lettre correspondante et on l'ajoute à la liste des prénoms
  lettre = liste_lettres[indice]
  prenoms_generes += lettre

  #On refait la séquence d'entrée du prédicteur
  sequence += lettre
  sequence = sequence[1:]

  #Si la lettre est le séparateur, on a nouveau nom
  if lettre == ' ':
    nombre_noms -= 1
    print("noms générés : ", 10 - nombre_noms)

prenoms = prenoms_generes.split()
print("Voici les 10 prénoms générés")
for prenom in prenoms :
  print(prenom.capitalize())

noms générés :  1
noms générés :  2
noms générés :  3
noms générés :  4
noms générés :  5
noms générés :  6
noms générés :  7
noms générés :  8
noms générés :  9
noms générés :  10
Voici les 10 prénoms générés
Oamhl
Earlic
Adre
R
Thée
Mazicc
Arriel
Théo
Maeva
Samy


In [ ]:
print(prenoms)

['oamhl', 'earlic', 'adre', 'r', 'thée', 'mazicc', 'arriel', 'théo', 'maeva', 'samy']


In [ ]:
liste_prenoms = ["Benjamin", "Danylo", "Paul", "Ethan", "Hazem", "Noé", "Raphaël", "Karliss", "Gabriel",
                 "Timothée","Soizic","Adrien","Théo","Maeva","Samy","Camille","Romain","Alexandre","Arthur"]

'a'

#Avec des listes de prénoms déjà faites

On va utiliser la liste de prénoms donnée ici : https://www.data.gouv.fr/fr/datasets/liste-de-prenoms/#resources

In [ ]:
import os

!wget "https://www.data.gouv.fr/fr/datasets/r/55cd803a-998d-4a5c-9741-4cd0ee0a7699"
os.rename("/content/55cd803a-998d-4a5c-9741-4cd0ee0a7699","/content/prenoms.csv" )

--2023-01-28 13:12:23--  https://www.data.gouv.fr/fr/datasets/r/55cd803a-998d-4a5c-9741-4cd0ee0a7699
Resolving www.data.gouv.fr (www.data.gouv.fr)... 37.59.183.93
Connecting to www.data.gouv.fr (www.data.gouv.fr)|37.59.183.93|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://static.data.gouv.fr/resources/liste-de-prenoms/20141127-154433/Prenoms.csv [following]
--2023-01-28 13:12:25--  https://static.data.gouv.fr/resources/liste-de-prenoms/20141127-154433/Prenoms.csv
Resolving static.data.gouv.fr (static.data.gouv.fr)... 37.59.183.93
Connecting to static.data.gouv.fr (static.data.gouv.fr)|37.59.183.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 265821 (260K) [text/csv]
Saving to: ‘55cd803a-998d-4a5c-9741-4cd0ee0a7699’

55cd803a-998d-4a5c- 100%[===================>] 259.59K   245KB/s    in 1.1s    

2023-01-28 13:12:27 (245 KB/s) - ‘55cd803a-998d-4a5c-9741-4cd0ee0a7699’ saved [265821/265821]



In [ ]:
import pandas as pd

#Attention, les caractères du csv sont codées en latin-1 et non en utf-8
#Bien évidemment, les créateurs n'ont pas utilisés une virgule comme séparateur mais un ;
data_prenoms = pd.read_csv("prenoms.csv", sep=";", encoding = 'latin-1')

data_prenoms.columns = ['prenom','genre','langage','frequence']
data_prenoms.sample(20)

,prenom,genre,langage,frequence
1010,aurora,f,roman mythology,0.29
6637,livia (1),f,"italian, portuguese, english, ancient roman",0.00
1121,banu,f,turkish,0.00
4371,hajna,f,hungarian,0.00
9795,shana,f,english,0.00
4099,glendower,m,welsh,0.00
6131,kondrat,m,polish,0.03
845,arkadi,m,russian,0.03
2228,costel,m,romanian,0.00
10859,ulric,m,english,0.00


In [ ]:
data_prenoms_sansna = data_prenoms.dropna()

data_prenoms_sansna[data_prenoms_sansna.prenom.str.contains("livia")]

,prenom,genre,langage,frequence
6637,livia (1),f,"italian, portuguese, english, ancient roman",0.00
6638,livia (2),f,english,0.00
6639,liviana,f,"italian, ancient roman",0.00
8243,olivia,f,english,1.27


In [ ]:
data_prenoms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11627 entries, 0 to 11626
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   prenom     11626 non-null  object 
 1   genre      11627 non-null  object 
 2   langage    11627 non-null  object 
 3   frequence  11627 non-null  float64
dtypes: float64(1), object(3)
memory usage: 363.5+ KB


In [ ]:
from tqdm import tqdm

#On remplit les NA de langage par 'french'
data_prenoms["langage"] = data_prenoms["langage"].fillna(value="french")

#On va récupérer seulement les noms qui ont comme langue 'french'
#data_prenoms_french = data_prenoms[data_prenoms.langage.str.contains('french')]

#si on veut sélectionner plusieurs langues
data_prenoms_ita_french = pd.concat([data_prenoms[data_prenoms.langage.str.contains('french')], data_prenoms[data_prenoms.langage.str.contains('italian')]])

#On récupère la colonne avec les prénoms
prenoms = data_prenoms_ita_french['prenom']

#On supprime les éléments vide avec dropna
prenoms = prenoms.dropna()

#on la convertit en liste
liste_prenoms = []

for prenom in tqdm(prenoms):
  #S'il y a une parenthèse, on ne garde que le prénom
  if '(' in prenom :
    prenom = prenom.split()[0]

  #On vérifie qu'il n'y ait pas de doublons
  if prenom not in liste_prenoms and '(' not in prenom :
    liste_prenoms.append(prenom)


100%|██████████| 1698/1698 [00:00<00:00, 67707.30it/s]


In [ ]:
print(len(liste_prenoms))

1692


In [ ]:
#si on veut sélectionner plusieurs langues
data_prenoms_ita_french = pd.concat([data_prenoms[data_prenoms.langage.str.contains('french')], data_prenoms[data_prenoms.langage.str.contains('italian')]])

data_prenoms_ita_french.sample(20)

,prenom,genre,langage,frequence
6403,lazare,m,french,4.43
3265,estachio,m,italian,0.00
4787,ianthina,f,french,0.00
11464,yvette,f,french,2.20
2075,clément,m,french,6.69
367,alfredo,m,"italian, spanish, portuguese",0.41
9089,régine,f,french,0.35
7274,marzio,m,italian,0.00
3254,esmée,f,french,0.96
3793,gaétan,m,french,0.03


In [ ]:
data_prenoms_ita_french.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1698 entries, 29 to 11572
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   prenom     1698 non-null   object 
 1   genre      1698 non-null   object 
 2   langage    1698 non-null   object 
 3   frequence  1698 non-null   float64
dtypes: float64(1), object(3)
memory usage: 66.3+ KB


##Mise en forme des données
On va faire une chaîne de caractère qui joint tous les éléments de la liste.

Pour ce faire, on va utiliser la méthode str.join(list)

In [ ]:
#On commence par chercher la longueur des séquences
longueur_sequence = 0

#Il va falloir mettre cette longueur par rapport au plus long nom
for prenom in liste_prenoms :
  longueur = len(prenom)

  #si la longueur du prénom est plus grande que celle de la séquence, on update cette dernière
  if longueur > longueur_sequence :
    longueur_sequence = longueur

#On ajoute 1 pour prendre en compte le séparateur ' '
longueur_sequence += 1

print("Longueur des séquences :", longueur_sequence)



Longueur des séquences : 14


In [ ]:
#On peut maintenant faire notre chaîne de prénoms :
chaine_prenoms = ' '.join(liste_prenoms).lower()

#On ajoute un espace à la fin
chaine_prenoms += ' '

print("Chaîne des prénoms :", chaine_prenoms)

Chaîne des prénoms : abel absolon achille adair adam adara adélaïde adèle adeline aden adena adolphe adonia adrien adrienne afra agathe aglea agnès aiden aimé aimée alain albert albertine aldrick alexandre alexandrie alexis alfred aliah alice aline alison alodia alphonse alphonsine alphonsus alta alva amara amarante amardad amaury ambre ambroise amédée amélie aminta amynta anaïs anastasie anatole andré andrée angèle angeline angelique anika anne annette ansel anselme antoine antoinette aphra apollinaire apolline aracelis aracely araminta aran ariane arianne arienne arik aristide arlen arlette armand armel armelle arnaud arnaude aryana ashanti ashtad athelstan atticus aude auguste augustin aurèle aurélie aurelien aurore avia avra avril axelle aylmer azarias azeneth balfour baptiste barbara barnabé barthélémy basile bastien baudouin béatrice beavis bénédicte benjamin benjamine benoit benoite berlin bernadette bernard bernardine berthe bertrand blaine blaise blanche boniface brannon brenn

On va faire une liste des différentes lettres et on pourra ainsi remplacer le caractère dans chaine_prenom par son indice.

In [ ]:
liste_lettres = []

#Pour chaque lettre de la chaine, on ajoute la lettre à liste_lettres si elle n'est pas déjà dedans
for lettre in chaine_prenoms :
  if lettre not in liste_lettres :
    liste_lettres.append(lettre)

print("Nombre de caractères différents :",len(liste_lettres))
print(liste_lettres)
print('indice de la lettre h :', liste_lettres.index('h'))

Nombre de caractères différents : 37
['a', 'b', 'e', 'l', ' ', 's', 'o', 'n', 'c', 'h', 'i', 'd', 'r', 'm', 'é', 'ï', 'è', 'p', 'f', 'g', 't', 'k', 'x', 'u', 'v', 'y', 'q', 'z', 'j', 'w', 'ç', '-', 'ô', 'ë', 'í', 'ì', 'ò']
indice de la lettre h : 9


On va pouvoir créer les séquences glissées sur la chaîne de prénoms

In [ ]:
#On va créer les séquences et la lettre suivante en même temps
sequences = []
lettres_suivantes = []

#On récupère la longueur de la chaîne de prénoms :
longueur_chaine = len(chaine_prenoms)

#on va faire une boucle sur des indices entre 0 et longueur_chaine - longueur_sequence :
for indice in range(0, longueur_chaine - longueur_sequence) :
  #on récupère une séquence dans chaîne nombre située entre indice et indice + longueur_sequence
  sequence = chaine_prenoms[indice : indice + longueur_sequence]
  #on récupère la lettre suivante :
  lettre_suivante = chaine_prenoms[indice + longueur_sequence]

  #On ajoute ces derniers éléments à nos 2 listes
  sequences.append(sequence)
  lettres_suivantes.append(lettre_suivante)

print("Nombre de séquences :", len(sequences))
#Vérification sur les 10 premiers éléments et le dernier
#On utilise enumerate() pour récupérer l'indice et l'élément de la liste avec for
for indice,sequence in enumerate(sequences[:10]):
  print("Séquence :", sequence, "    Lettre suivante :",lettres_suivantes[indice])

print()
print("Dernières occurences :")
print("Séquence :", sequences[-1], "    Lettre suivante :",lettres_suivantes[-1])

Nombre de séquences : 13020
Séquence : abel absolon a     Lettre suivante : c
Séquence : bel absolon ac     Lettre suivante : h
Séquence : el absolon ach     Lettre suivante : i
Séquence : l absolon achi     Lettre suivante : l
Séquence :  absolon achil     Lettre suivante : l
Séquence : absolon achill     Lettre suivante : e
Séquence : bsolon achille     Lettre suivante :  
Séquence : solon achille      Lettre suivante : a
Séquence : olon achille a     Lettre suivante : d
Séquence : lon achille ad     Lettre suivante : a

Dernières occurences :
Séquence : iana zara zita     Lettre suivante :  


## Création des X, Y

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

#On stocke la longueur de liste_lettres
categories = len(liste_lettres)

#on va commencer par créer les X
X=[]

#on crée une boucle sur les séquences
for sequence in sequences :
  #on va créer une liste vide pour récolter les indices des caractères
  liste = []
  for caractere in sequence :
    #on récupère l'indice correspondant au caractère
    indice = liste_lettres.index(caractere)
    #on ajoute l'indice à la liste
    liste.append(indice)

  #on convertit tout de suite avec to_categorical pour avoir un vecteur de proba de 26 éléments
  liste = to_categorical(liste,categories)
  #On ajoute la liste dans X
  X.append(liste)

X= np.asarray(X)
print(X.shape)

#pour les Y, on ne change pas de méthode
#il faut quand même convertir les caractères en indice
Y=[]
for caractere in lettres_suivantes :
  indice = liste_lettres.index(caractere)
  Y.append(indice)

#to_categorical
Y = to_categorical(Y,categories)

print(Y.shape)

(13020, 14, 37)
(13020, 37)


##Réseau de neurones avec LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

modele = Sequential()

#On ajoute en entrée une couche de neurones LSTM
modele.add(LSTM(64, input_shape = (longueur_sequence,categories), recurrent_dropout = 0.2))

#On ajoute la sortie
modele.add(Dense(categories, activation='softmax'))

modele.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                26112     
                                                                 
 dense_1 (Dense)             (None, 37)                2405      
                                                                 
Total params: 28,517
Trainable params: 28,517
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import RMSprop

optimizer = RMSprop(learning_rate = 0.01)

modele.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',])

modele.fit(X, Y, batch_size = 32, epochs = 50)

modele.save('name_generator.h5')

Epoch 1/50
407/407 [==============================] - 9s 17ms/step - loss: 2.4346 - accuracy: 0.2618
Epoch 2/50
407/407 [==============================] - 7s 18ms/step - loss: 2.0517 - accuracy: 0.3714
Epoch 3/50
407/407 [==============================] - 7s 17ms/step - loss: 1.8049 - accuracy: 0.4504
Epoch 4/50
407/407 [==============================] - 7s 17ms/step - loss: 1.6554 - accuracy: 0.5018
Epoch 5/50
407/407 [==============================] - 7s 17ms/step - loss: 1.5531 - accuracy: 0.5280
Epoch 6/50
407/407 [==============================] - 7s 17ms/step - loss: 1.4860 - accuracy: 0.5492
Epoch 7/50
407/407 [==============================] - 7s 17ms/step - loss: 1.4364 - accuracy: 0.5660
Epoch 8/50
407/407 [==============================] - 7s 17ms/step - loss: 1.3926 - accuracy: 0.5743
Epoch 9/50
407/407 [==============================] - 7s 17ms/step - loss: 1.3600 - accuracy: 0.5843
Epoch 10/50
407/407 [==============================] - 7s 17ms/step - loss: 1.3267 - accura

KeyboardInterrupt: ignored

##Création du prédicteur

In [ ]:
#On va décider de la première lettre :
lettre = 'na'
#lettre = np.random.choice(liste_lettres)

lettres_choisies = lettre

#On récupère la dernière séquence ou une séquence de même taille de notre choix
#sequence="a zvonko zygmunt"
sequence="iana zara zita"

#On ajoute à la fin un caractère espace et la lettre choisie et on supprime les 2 premiers caractères
sequence += ' ' + lettre
sequence = sequence[1+len(lettre):]

#print(len(sequence),sequence)

#On crée une nouvelle chaine pour stocker les prénoms générés en commençant par la lettre choisie :
prenoms_generes = lettre

#On donne combien de noms vont être générés :
nombre_noms = 10

#On prend un compteur de caractères
nombre_lettres = 0

while nombre_noms > 0 :
  X = []

  #on recrée X
  for caractere in sequence :
    indice = liste_lettres.index(caractere)
    X.append(indice)

  #On to_categorical X
  X = to_categorical(X, categories)

  #il faut reshape X pour avoir un tenseur en 3 dimensions
  X = np.reshape(X, (1,X.shape[0],X.shape[1]))
  #print(X.shape)

  #On prédit le vecteur de probabilités
  probas = modele.predict(X, verbose = 0)
  probas = probas[0]

  #On choisit aléatoirement l'indice parmi les catégories en utilisant la distribution calculée
  indice = np.random.choice(categories, p=probas)

  #On ajoute au nombre de lettres dans le prénom
  nombre_lettres += 1

  #On récupère la lettre correspondante et on l'ajoute à la liste des prénoms
  lettre = liste_lettres[indice]
  prenoms_generes += lettre

  #On refait la séquence d'entrée du prédicteur
  sequence += lettre
  sequence = sequence[1:]

  #Si la lettre est le séparateur, on a nouveau nom
  if lettre == ' ' and nombre_lettres > 3:
    nombre_noms -= 1
    print("noms générés : ", 10 - nombre_noms)
    #remise à 0 du nombre de lettres
    nombre_lettres = 0


  #On refait la séquence pour commencer par les lettres choisies
  if lettre == ' ':
    sequence += lettres_choisies
    sequence = sequence[len(lettres_choisies):]
    #On réinitialise le début du prénom
    prenoms_generes += lettres_choisies


prenoms = prenoms_generes.split()
print("Voici les 10 prénoms générés")
for prenom in prenoms :
  if len(prenom) >= 3 and '(' not in prenom:
    print(prenom.capitalize())

noms générés :  1
noms générés :  2
noms générés :  3
noms générés :  4
noms générés :  5
noms générés :  6
noms générés :  7
noms générés :  8
noms générés :  9
noms générés :  10
Voici les 10 prénoms générés
Nardo
Natala
Nataniel
Natalio
Natalina
Naéline
Nannine
Nadtaes
Natheline
Nauenne


#Avec des Données récupérées sur Kaggle
https://www.kaggle.com/

On va récupérer les données d'un kaggle Pokemon : https://www.kaggle.com/datasets/abcsds/pokemon

##Affilier le Dataset de Kaggle à notre notebook

On va suivre ce tuto (sauf la partie Google Drive qui ralentit l'importation d'image) : [Tuto Kaggle](https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a)

1.   Se connecter sur [Kaggle](https://www.kaggle.com/)
2.   On va sur Account pour expirer notre jeton API et en créer un nouveau
3.   On importe le fichier kaggle.json sur notre notebook
4.   On importe "l'environnement Kaggle" avec le code ci-dessous (on enlève toute référence à Drive par rapport au tuto)

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

5.   On va récupérer la commande API du dataset (en cliquant sur les 3 petits points à droite)
6.   On met dans le code un point d'exclamation ! puis on colle la commande du dataset.
7.   Il télécharge ensuite un fichier zip contenant le dataset dans nos fichiers.

In [ ]:
!kaggle datasets download -d abcsds/pokemon

  0% 0.00/14.9k [00:00<?, ?B/s]
100% 14.9k/14.9k [00:00<00:00, 10.4MB/s]


8.   On va dézipper l'archive

In [ ]:
!unzip \*.zip

Archive:  pokemon.zip
  inflating: Pokemon.csv             


In [ ]:
!rm \*.zip

rm: cannot remove '*.zip': No such file or directory


9.   ça y est notre Dataset est importé dans les fichiers

##Récupération du CSV

In [ ]:
import pandas as pd

data_pokemon = pd.read_csv("Pokemon.csv")

#data_prenoms.columns = ['prenom','genre','langage','frequence']
data_pokemon.sample(20)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
408,373,Salamence,Dragon,Flying,600,95,135,80,110,80,100,3,False
129,120,Staryu,Water,NaN,340,30,45,55,70,55,85,1,False
147,136,Flareon,Fire,NaN,525,65,130,60,95,110,65,1,False
788,712,Bergmite,Ice,NaN,304,55,69,85,32,35,28,6,False
273,253,Grovyle,Grass,NaN,405,50,65,45,85,65,95,3,False
479,431,Glameow,Normal,NaN,310,49,55,42,42,37,85,4,False
665,604,Eelektross,Electric,NaN,515,85,115,80,105,80,50,5,False
15,12,Butterfree,Bug,Flying,395,60,45,50,90,80,70,1,False
128,119,Seaking,Water,NaN,450,80,92,65,65,80,68,1,False
332,306,Aggron,Steel,Rock,530,70,110,180,60,60,50,3,False


In [ ]:
data_pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


In [ ]:
from tqdm import tqdm

#On récupère la colonne avec les prénoms
prenoms = data_pokemon['Name']

#On supprime les éléments vide avec dropna
prenoms = prenoms.dropna()

#on la convertit en liste
liste_prenoms = []

for prenom in tqdm(prenoms):
  #On vérifie qu'il n'y ait pas de doublons
  if prenom not in liste_prenoms and 'Mega' not in prenom and 'Primal' not in prenom and ' X' not in prenom and ' Y' not in prenom :
    liste_prenoms.append(prenom)


100%|██████████| 800/800 [00:00<00:00, 94775.82it/s]


In [ ]:
print(len(liste_prenoms))

749


##Mise en forme des données
On va faire une chaîne de caractère qui joint tous les éléments de la liste.

Pour ce faire, on va utiliser la méthode str.join(list)

In [ ]:
#On commence par chercher la longueur des séquences
longueur_sequence = 0

#Il va falloir mettre cette longueur par rapport au plus long nom
for prenom in liste_prenoms :
  longueur = len(prenom)

  #si la longueur du prénom est plus grande que celle de la séquence, on update cette dernière
  if longueur > longueur_sequence :
    longueur_sequence = longueur

#On ajoute 1 pour prendre en compte le séparateur ' '
longueur_sequence += 1

print("Longueur des séquences :", longueur_sequence)



Longueur des séquences : 25


In [ ]:
#On peut maintenant faire notre chaîne de prénoms :
chaine_prenoms = ' '.join(liste_prenoms).lower()

#On ajoute un espace à la fin
chaine_prenoms += ' '

print("Chaîne des prénoms :", chaine_prenoms)

Chaîne des prénoms : bulbasaur ivysaur venusaur charmander charmeleon charizard squirtle wartortle blastoise caterpie metapod butterfree weedle kakuna beedrill pidgey pidgeotto pidgeot rattata raticate spearow fearow ekans arbok pikachu raichu sandshrew sandslash nidoran♀ nidorina nidoqueen nidoran♂ nidorino nidoking clefairy clefable vulpix ninetales jigglypuff wigglytuff zubat golbat oddish gloom vileplume paras parasect venonat venomoth diglett dugtrio meowth persian psyduck golduck mankey primeape growlithe arcanine poliwag poliwhirl poliwrath abra kadabra alakazam machop machoke machamp bellsprout weepinbell victreebel tentacool tentacruel geodude graveler golem ponyta rapidash slowpoke slowbro magnemite magneton farfetch'd doduo dodrio seel dewgong grimer muk shellder cloyster gastly haunter gengar onix drowzee hypno krabby kingler voltorb electrode exeggcute exeggutor cubone marowak hitmonlee hitmonchan lickitung koffing weezing rhyhorn rhydon chansey tangela kangaskhan horsea s

On va faire une liste des différentes lettres et on pourra ainsi remplacer le caractère dans chaine_prenom par son indice.

In [ ]:
liste_lettres = []

#Pour chaque lettre de la chaine, on ajoute la lettre à liste_lettres si elle n'est pas déjà dedans
for lettre in chaine_prenoms :
  if lettre not in liste_lettres :
    liste_lettres.append(lettre)

print("Nombre de caractères différents :",len(liste_lettres))
print(liste_lettres)
print('indice de la lettre h :', liste_lettres.index('h'))

Nombre de caractères différents : 37
['b', 'u', 'l', 'a', 's', 'r', ' ', 'i', 'v', 'y', 'e', 'n', 'c', 'h', 'm', 'd', 'o', 'z', 'q', 't', 'w', 'p', 'f', 'k', 'g', '♀', '♂', 'x', 'j', "'", '.', '2', '-', 'é', '5', '0', '%']
indice de la lettre h : 13


On va pouvoir créer les séquences glissées sur la chaîne de prénoms

In [ ]:
#On va créer les séquences et la lettre suivante en même temps
sequences = []
lettres_suivantes = []

#On récupère la longueur de la chaîne de prénoms :
longueur_chaine = len(chaine_prenoms)

#on va faire une boucle sur des indices entre 0 et longueur_chaine - longueur_sequence :
for indice in range(0, longueur_chaine - longueur_sequence) :
  #on récupère une séquence dans chaîne nombre située entre indice et indice + longueur_sequence
  sequence = chaine_prenoms[indice : indice + longueur_sequence]
  #on récupère la lettre suivante :
  lettre_suivante = chaine_prenoms[indice + longueur_sequence]

  #On ajoute ces derniers éléments à nos 2 listes
  sequences.append(sequence)
  lettres_suivantes.append(lettre_suivante)

print("Nombre de séquences :", len(sequences))
#Vérification sur les 10 premiers éléments et le dernier
#On utilise enumerate() pour récupérer l'indice et l'élément de la liste avec for
for indice,sequence in enumerate(sequences[:10]):
  print("Séquence :", sequence, "    Lettre suivante :",lettres_suivantes[indice])

print()
print("Dernières occurences :")
print("Séquence :", sequences[-1], "    Lettre suivante :",lettres_suivantes[-1])

Nombre de séquences : 6775
Séquence : bulbasaur ivysaur venusau     Lettre suivante : r
Séquence : ulbasaur ivysaur venusaur     Lettre suivante :  
Séquence : lbasaur ivysaur venusaur      Lettre suivante : c
Séquence : basaur ivysaur venusaur c     Lettre suivante : h
Séquence : asaur ivysaur venusaur ch     Lettre suivante : a
Séquence : saur ivysaur venusaur cha     Lettre suivante : r
Séquence : aur ivysaur venusaur char     Lettre suivante : m
Séquence : ur ivysaur venusaur charm     Lettre suivante : a
Séquence : r ivysaur venusaur charma     Lettre suivante : n
Séquence :  ivysaur venusaur charman     Lettre suivante : d

Dernières occurences :
Séquence : pahoopa unbound volcanion     Lettre suivante :  


## Création des X, Y

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

#On stocke la longueur de liste_lettres
categories = len(liste_lettres)

#on va commencer par créer les X
X=[]

#on crée une boucle sur les séquences
for sequence in sequences :
  #on va créer une liste vide pour récolter les indices des caractères
  liste = []
  for caractere in sequence :
    #on récupère l'indice correspondant au caractère
    indice = liste_lettres.index(caractere)
    #on ajoute l'indice à la liste
    liste.append(indice)

  #on convertit tout de suite avec to_categorical pour avoir un vecteur de proba de 26 éléments
  liste = to_categorical(liste,categories)
  #On ajoute la liste dans X
  X.append(liste)

X= np.asarray(X)
print(X.shape)

#pour les Y, on ne change pas de méthode
#il faut quand même convertir les caractères en indice
Y=[]
for caractere in lettres_suivantes :
  indice = liste_lettres.index(caractere)
  Y.append(indice)

#to_categorical
Y = to_categorical(Y,categories)

print(Y.shape)

(6775, 25, 37)
(6775, 37)


##Réseau de neurones avec LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

modele = Sequential()

#On ajoute en entrée une couche de neurones LSTM
modele.add(LSTM(64, input_shape = (longueur_sequence,categories), recurrent_dropout = 0.2))

#On ajoute la sortie
modele.add(Dense(categories, activation='softmax'))

modele.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 64)                26112     
                                                                 
 dense_2 (Dense)             (None, 37)                2405      
                                                                 
Total params: 28,517
Trainable params: 28,517
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import RMSprop

optimizer = RMSprop(learning_rate = 0.01)

modele.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',])

modele.fit(X, Y, batch_size = 32, epochs = 50)

modele.save('name_generator.h5')

Epoch 1/50
212/212 [==============================] - 51s 29ms/step - loss: 2.8331 - accuracy: 0.1634
Epoch 2/50
212/212 [==============================] - 6s 29ms/step - loss: 2.5554 - accuracy: 0.2303
Epoch 3/50
212/212 [==============================] - 6s 29ms/step - loss: 2.4583 - accuracy: 0.2517
Epoch 4/50
212/212 [==============================] - 6s 29ms/step - loss: 2.3819 - accuracy: 0.2729
Epoch 5/50
212/212 [==============================] - 6s 29ms/step - loss: 2.3083 - accuracy: 0.2955
Epoch 6/50
212/212 [==============================] - 6s 29ms/step - loss: 2.2381 - accuracy: 0.3144
Epoch 7/50
212/212 [==============================] - 6s 28ms/step - loss: 2.1858 - accuracy: 0.3293
Epoch 8/50
212/212 [==============================] - 6s 29ms/step - loss: 2.1201 - accuracy: 0.3457
Epoch 9/50
212/212 [==============================] - 6s 29ms/step - loss: 2.0702 - accuracy: 0.3528
Epoch 10/50
212/212 [==============================] - 6s 29ms/step - loss: 2.0294 - accur

KeyboardInterrupt: ignored

##Création du prédicteur

In [ ]:
#On va décider de la première lettre :
lettre = 'na'
#lettre = np.random.choice(liste_lettres)

lettres_choisies = lettre

#On récupère la dernière séquence ou une séquence de même taille de notre choix
#sequence="a zvonko zygmunt"
sequence="pahoopa unbound volcanion"

#On ajoute à la fin un caractère espace et la lettre choisie et on supprime les 2 premiers caractères
sequence += ' ' + lettre
sequence = sequence[1+len(lettre):]

#print(len(sequence),sequence)

#On crée une nouvelle chaine pour stocker les prénoms générés en commençant par la lettre choisie :
prenoms_generes = lettre

#On donne combien de noms vont être générés :
nombre_noms = 10

#On prend un compteur de caractères
nombre_lettres = 0

while nombre_noms > 0 :
  X = []

  #on recrée X
  for caractere in sequence :
    indice = liste_lettres.index(caractere)
    X.append(indice)

  #On to_categorical X
  X = to_categorical(X, categories)

  #il faut reshape X pour avoir un tenseur en 3 dimensions
  X = np.reshape(X, (1,X.shape[0],X.shape[1]))
  #print(X.shape)

  #On prédit le vecteur de probabilités
  probas = modele.predict(X, verbose = 0)
  probas = probas[0]

  #On choisit aléatoirement l'indice parmi les catégories en utilisant la distribution calculée
  indice = np.random.choice(categories, p=probas)

  #On ajoute au nombre de lettres dans le prénom
  nombre_lettres += 1

  #On récupère la lettre correspondante et on l'ajoute à la liste des prénoms
  lettre = liste_lettres[indice]
  prenoms_generes += lettre

  #On refait la séquence d'entrée du prédicteur
  sequence += lettre
  sequence = sequence[1:]

  #Si la lettre est le séparateur, on a nouveau nom
  if lettre == ' ' and nombre_lettres > 3:
    nombre_noms -= 1
    print("noms générés : ", 10 - nombre_noms)
    #remise à 0 du nombre de lettres
    nombre_lettres = 0


  #On refait la séquence pour commencer par les lettres choisies
  if lettre == ' ':
    sequence += lettres_choisies
    sequence = sequence[len(lettres_choisies):]
    #On réinitialise le début du prénom
    prenoms_generes += lettres_choisies


prenoms = prenoms_generes.split()
print("Voici les 10 prénoms générés")
for prenom in prenoms :
  if len(prenom) >= 3 and '(' not in prenom:
    print(prenom.capitalize())

noms générés :  1
noms générés :  2
noms générés :  3
noms générés :  4
noms générés :  5
noms générés :  6
noms générés :  7
noms générés :  8
noms générés :  9
noms générés :  10
Voici les 10 prénoms générés
Nanccino
Nandave
Nadosmpla
Naudoninash
Nadotonz
Naincoke
Naunchan
Nancsee
Nanserin
Nangada
